In [98]:
import pandas as pd


In [99]:
df=pd.read_csv('spotify_millsongdata.csv')

In [100]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [101]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [102]:
df.shape

(57650, 4)

In [103]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [104]:
df=df.sample(5000)

In [105]:
df=df.drop('link',axis=1).reset_index(drop=True)

In [106]:
df.head(10)

,artist,song,text
0,Katy Perry,Trust In Me,How could I see you when I was so blind \r\nH...
1,Fall Out Boy,Moving Pictures,Last night I saw a movie \r\nAnd I thought ab...
2,Perry Como,"Goodnight, Sweet Jesus","Goodnight, Sweet Jesus, \r\nGoodnight, goodni..."
3,Reba Mcentire,Nobody Dies From A Broken Heart,Nobody told me \r\nIt could ever hurt so bad ...
4,Squeeze,His House Her Home,I think to myself when we kiss \r\nYour husba...
5,Richard Marx,Shine,The tree you planted \r\nIs still holding on ...
6,Bonnie Raitt,Finest Lovin' Man,The man I'm lovin' don't worry 'bout me runnin...
7,Death,Secret Face,There is a mask \r\nThat covers up one's true...
8,Chris Rea,Hired Gun,Morning light meet the day \r\nNervous eyes l...
9,Pet Shop Boys,We All Feel Better In The Dark,"The music's pumping through me, I rush into th..."


In [107]:
df.shape

(5000, 3)

Text Cleaning/ Text Processing

In [108]:
df['text']=df['text'].str.lower().replace(r'\W\S',' ').replace(r'\n',' ',regex=True)

In [109]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xannp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [111]:
tokenization("you are beauty")

'you are beauti'

In [112]:
df['text']=df['text'].apply(lambda x:token(x))

In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [114]:
tfid=TfidfVectorizer(analyzer="word",stop_words='english')

In [115]:
matrix=tfid.fit_transform(df['text'])

In [116]:
similar=cosine_similarity(matrix)

In [117]:
similar[0]

array([1., 0., 0., ..., 0., 0., 0.])

In [118]:
df.tail(5)

,artist,song,text
4995,Fleetwood Mac,Hi Ho Silver,comeinthihousstopallofthatyaketiyakcomeinthiho...
4996,Hank Williams Jr.,Honky Tonkin',whenyouaresadandloneandhavenoplacetogocallmeup...
4997,Harry Belafonte,Come Back Liza,"whenevi'mawayfromliza,watercometomeeyeeveritim..."
4998,Pretenders,Baby's Breath,therewaatimeyouthoughtiwainlovewithyouwellthin...
4999,Tragically Hip,Three Pistols,tomthompsoncamepaddlpasti'mprettisureitwahiman...


In [119]:
df[df['song']=='Diamond Joe'].index[0]

1387

Recommender System

In [120]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similar[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs
        

In [122]:
recommendation("Shine")

['The Pilgrim',
 'Arms Open Wide',
 'Born To Die',
 'Bananas To The Beat',
 'Island In The Sun',
 'Young Without Lovers',
 'Candyman',
 'Speedball Tucker',
 'Try A Little Harder',
 'What Are You Waiting For',
 'Moment of Surrender',
 "Beer Drinker's Waltz",
 'Just Around The Corner',
 'Shining Star',
 'Run Runaway',
 'Love To Hate',
 'Dope Nose',
 'The Righteous And The Wicked',
 'Rock Me Amadeus',
 'Pictures Of Home']

In [123]:
import pickle
pickle.dump(similar,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))